In [45]:
import json
import os

from words import *
from top_dense import *
from wordbank import *
from seedlings import *

import networkx as nx
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly.graph_objs import *


In [33]:
%reload_ext autoreload
%autoreload 2

%matplotlib inline

## Load the word vectors

We're using the 42 billion token, 300D, GloVe model, 
(glove.42B.300d.zip from http://nlp.stanford.edu/projects/glove/)
with 1.9 million words in the vocab. The original 
vector file was parsed into a numpy array and that 
numpy array was saved to disc. The function that does
this parsing/loading is words.create_numpy_from_glove()
We load that numpy array and the vocab dictionary from
the output of that function here.

In [5]:
glove = GloVe(vocab="data/model/dict_glove_42b_300", 
              vectors="data/model/vectors_glove_42b_300.npy")

### Load the WordBank data

Load the Wordbank data. This dataset is a measure of words currently in the infants' vocabulary. We filter these words for just nouns, so we can compare it with the SEEDLings data. We use the "category" and "type" fields for each word to determine this. These words will be passed into the semantic graph generator. We'll do the same thing to the environmental linguistic input using the SEEDLings corpus.

The resulting wordmap object is a dictionary of keys to words. Here key is just the name of the input dataset, 
i.e. wordbank_english

In [15]:
wordbank_english = WordBank(input="data/wb_cdi/wordbank_english.csv")

wb_eng_wordmap = wordbank_english.wordmap()

### Load SEEDLings Monthly Data

We load the aggregated monthly words from the SEEDLings corpus. Each month is its own dataset (i.e. a key in the wordmap)

In [21]:
seedlings_wordmap = load_seedlings("data/seedlings/bl")

### Generate semantic graphs

For every word, give it an edge with any other word that's within some threshold of cos(θ) to it (i.e.  cos(θ) >= 1.0 - threshold ). Output these graphs in JSON form with each word as a key, and a list of its neighbors as the value. We're generating a distinct graph for a range of threshold values. The "start" and "end" parameters specify this range and the "step" parameter specifies the granularity. 

In [16]:
# English WordBank
glove.graph_cosine_range(output_path="english_wordbank", wordmap=wb_eng_wordmap, 
                         start=0.3, end=0.81, step=0.01)

In [ ]:
# monthly aggregated SEEDLings
glove.graph_cosine_range(output_path="seedlings", wordmap=seedlings_wordmap, 
                         start=0.3, end=0.81, step=0.01)

### Rank nodes in each graph by their degree

For each of the graphs that we generated in the previous step, we output a csv file of the top ranked words (by degree). 

In [17]:
rank_density(input_path="data/output/english_wordbank/semgraphs", 
             output_path="data/output/english_wordbank/ranked_out")

In [ ]:
rank_density(input_path="data/output/seedlings/semgraphs", 
             output_path="data/output/seedlings/ranked_out")

In [67]:
init_notebook_mode(connected=True)
with open("data/output/english_wordbank/semgraphs/cosine_0.6/semgraph_wordbank_english", "rU") as input:
    semgraph = json.load(input)
    
N = len(semgraph)
G = nx.MultiGraph()

for key in semgraph:
    G.add_node(key, word=key, size=1)

for key, value in semgraph.items():
    if value:
        for element in value: 
            G.add_edge(key, element[0])

            
pos=nx.fruchterman_reingold_layout(G) 
nx.set_node_attributes(G, 'pos', pos)


edge_trace = Scatter(
    x=[], 
    y=[], 
    line=Line(width=0.5,color='#888'),
    hoverinfo='none',
    mode='lines')

for edge in G.edges():
    x0, y0 = G.node[edge[0]]['pos']
    x1, y1 = G.node[edge[1]]['pos']
    edge_trace['x'] += [x0, x1, None]
    edge_trace['y'] += [y0, y1, None]

node_trace = Scatter(
    x=[], 
    y=[], 
    text=[],
    mode='markers', 
    hoverinfo='text',
    marker=Marker(
        showscale=True,
        # colorscale options
        # 'Greys' | 'Greens' | 'Bluered' | 'Hot' | 'Picnic' | 'Portland' |
        # Jet' | 'RdBu' | 'Blackbody' | 'Earth' | 'Electric' | 'YIOrRd' | 'YIGnBu'
        colorscale='Portland',
        reversescale=True,
        color=[], 
        size=10,         
        colorbar=dict(
            thickness=15,
            title='Node Connections',
            xanchor='left',
            titleside='right'
        ),
        line=dict(width=2)))

for node in G.nodes():
    the_node = G.node[node]
    x, y = G.node[node]['pos']
    node_trace['x'].append(x)
    node_trace['y'].append(y)
    node_trace['marker']['color'].append(len(G.neighbors(node)))
    node_trace['text'].append('{}<br># connections: {}'.format(the_node['word'],len(G.neighbors(node))))
    
    
    

fig = Figure(data=Data([edge_trace, node_trace]),
             layout=Layout(
                title='<br>Semantic Graph',
                titlefont=dict(size=16),
                showlegend=False, 
                width=650,
                height=650,
                hovermode='closest',
                margin=dict(b=20,l=5,r=5,t=40),
                annotations=[ dict(
                     text="something", #Python code: <a href='https://plot.ly/ipython-notebooks/network-graphs/'> https://plot.ly/ipython-notebooks/network-graphs/</a>",
                    showarrow=False,
                    xref="paper", yref="paper",
                    x=0.005, y=-0.002 ) ],
                xaxis=XAxis(showgrid=False, zeroline=False, showticklabels=False),
                yaxis=YAxis(showgrid=False, zeroline=False, showticklabels=False)))


iplot(fig, filename='semantic_graph')


    